# Neural Particles 2D

## 2D Dataset

### Setup

In [4]:
import os
from subprocess import Popen, PIPE

manta_loc = "2D_SPH/"

data_version = 0
data_loc = "2D_data/"
prefix = "sph_2D"
src = "lowres"
ref = "highres"

def createFolder(p):
    if not os.path.exists(p):
        os.makedirs(p)
    
createFolder(data_loc)
createFolder(data_loc+"/"+src)
createFolder(data_loc+"/"+ref)
createFolder(data_loc+"/patches")
createFolder(data_loc+"/patches/"+src)
createFolder(data_loc+"/patches/"+ref)

def run_manta(scene, param={}):
    command = [manta_loc+"build/manta", manta_loc+scene]

    for k, v in param.items():
        command += [k, str(v)]
        
    print(" ".join(command) + "\n")

    proc = Popen(command, stdin=None, stdout=PIPE, stderr=PIPE)

    for line in proc.stdout:
        print(line.decode('utf-8'))
    for line in proc.stderr:
        print(line.decode('utf-8'))
    

### Generate High-res Data

In [6]:
import random

ref_prefix = "%s/%s/ref_%s_v%02d" % (data_loc, ref, prefix, data_version)

frame_count = 50
sub_res = 2
res = 150

# count of training/validation setups
data_count = 1#30

# param for data generation
seed = 124820112
min_scale = 0.05
max_scale = 0.3
min_pos = 0.1
max_pos = 0.9
max_h = 0.2
max_cnt = 5

param = {}

#disable gui
param['gui'] = 1

# resolution of domain
param['res'] = res
param['sres'] = sub_res

# output file format
param['out'] = ref_prefix

# write only every 30th frame -> 30 frames are one timestep
param['fps'] = 30

# simulation time (how many frames)
param['t_end'] = float(frame_count) / param['fps']

# run random training setups
random.seed(seed)
for i in range(data_count):
    param['out'] = (ref_prefix + "_d%03d")%i + "_%03d"
    
    # generate different cubes with dataformat "pos_x,pos_y,scale_x,scale_y"
    param['c_cnt'] = random.randint(1,max_cnt)
    cubes = {}
    for c in range(param['c_cnt']):    
        scx = random.uniform(min_scale, max_scale)
        scy = random.uniform(min_scale, max_scale)
        px = random.uniform(min_pos+scx/2, max_pos-scx/2)
        py = random.uniform(0, max_h) + scy/2
        print("%f,%f,%f,%f"%(px,py,scx,scy))
        cubes['c%d'%c] = "%f,%f,%f,%f"%(px,py,scx,scy)
    run_manta("scenes/2D_sph.py", dict(param, **cubes))

0.245092,0.179315,0.145915,0.178643
0.567569,0.178842,0.131005,0.299544
0.519023,0.091649,0.077867,0.098838
0.730030,0.200605,0.062978,0.062918
2D_SPH/build/manta 2D_SPH/scenes/2D_sph.py res 150 c0 0.245092,0.179315,0.145915,0.178643 sres 2 fps 30 out 2D_data//highres/ref_sph_2D_v00_d000_%03d c3 0.730030,0.200605,0.062978,0.062918 c_cnt 4 t_end 1.6666666666666667 gui 1 c2 0.519023,0.091649,0.077867,0.098838 c1 0.567569,0.178842,0.131005,0.299544

Version: mantaflow mac 64bit fp1 commit f6cd53f8cf8363306de2413f0be13c2338d6b62d from Oct 19 2017, 11:11:57

Loading script '2D_SPH/scenes/2D_sph.py'

Added & initialized 9344 particles

Added & initialized 20416 particles

   Frame 0, dt: 0.0006420951, 0/0.03333334 lock:0

   Frame 0, dt: 0.0006420951, 0.0006420951/0.03333334 lock:0

writing particles pp to uni file 2D_data//highres/ref_sph_2D_v00_d000_000_ps.uni

writing particle data pT to uni file 2D_data//highres/ref_sph_2D_v00_d000_000_pt.uni

writing particle data pV to uni file 2D_data

### Generate Low-res Data
Generate low-res data by down-scaling the high-res data by a given factor:
$$lowres=\frac{res}{\sqrt(factor)}$$

In [20]:
import math

src_prefix = "%s/%s/%s_v%02d" % (data_loc, src, prefix, data_version)

# downscale factor
factor = 10

param = {}

param['res'] = res
param['sres'] = sub_res
 
param['factor'] = factor
param['gui'] = 1
param['t'] = frame_count

for i in range(data_count):
    param['in'] = (ref_prefix + "_d%03d")%i + "_%03d"
    param['out'] = (src_prefix + "_d%03d")%i + "_%03d"
    run_manta("scenes/down_scale.py", param)

2D_SPH/build/manta 2D_SPH/scenes/down_scale.py res 150 t 50 out 2D_data//lowres/sph_2D_v00_d000_%03d factor 10 in 2D_data//highres/ref_sph_2D_v00_d000_%03d sres 2 gui 1

Version: mantaflow mac 64bit fp1 commit f6cd53f8cf8363306de2413f0be13c2338d6b62d from Oct 19 2017, 11:11:57

Loading script '2D_SPH/scenes/down_scale.py'

reading particles pp from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_ps.uni

reading particle data pT from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_pt.uni

reading particle data pV from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_pv.uni

reading particle data pD from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_pd.uni

reading particle data pP from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_pp.uni

reading particles high_pp from uni file 2D_data//highres/ref_sph_2D_v00_d000_000_ps.uni

 Deleted 26295 particles

writing particles pp to uni file 2D_data//lowres/sph_2D_v00_d000_000_ps.uni

Writing grid  to uni file 2D_data//lowres/sph_2

### Extract Surface-Patches

In [21]:
src_patches_path = "%s/patches/%s/%s_v%02d" % (data_loc, src, prefix, data_version)
ref_patches_path = "%s/patches/%s/ref_%s_v%02d" % (data_loc, ref, prefix, data_version)

param = {}

param["t"] = frame_count

# patch size
param["psize"] = 9
param["stride"] = 3

# tolerance of surface
param["surface"] = 0.5

for i in range(data_count):
    param["h_in"] = (ref_prefix + "_d%03d")%i + "_%03d"
    param["l_in"] = (src_prefix + "_d%03d")%i + "_%03d"
    param["h_out"] = (ref_patches_path + "_d%03d")%i + "_%03d"
    param["l_out"] = (src_patches_path + "_d%03d")%i + "_%03d"
    run_manta("scenes/extract_patches.py", param)

2D_SPH/build/manta 2D_SPH/scenes/extract_patches.py psize 9 h_out 2D_data//patches/highres/ref_sph_2D_v00_d000_%03d l_out 2D_data//patches/lowres/sph_2D_v00_d000_%03d l_in 2D_data//lowres/sph_2D_v00_d000_%03d t 50 h_in 2D_data//highres/ref_sph_2D_v00_d000_%03d stride 3

Version: mantaflow mac 64bit fp1 commit f6cd53f8cf8363306de2413f0be13c2338d6b62d from Oct 19 2017, 11:11:57

Loading script '2D_SPH/scenes/extract_patches.py'

Script finished.



## Train Neural Network

### Setup Neural Network

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

conv_f = 3
conv_k = 3
conv_s = (1, 1)

deconv_f = 3
deconv_k = 3
deconv_s = (1, 1)

input_shape = (64, 64, 1)

model = Sequential()

model.add(keras.layers.Conv2D(input_shape=input_shape, filters=conv_f,            \
                              kernel_size=conv_k, strides=conv_s,                 \
                              padding='valid', data_format=None,                  \
                              dilation_rate=(1, 1), activation=None,              \
                              use_bias=True, kernel_initializer='glorot_uniform', \
                              bias_initializer='zeros', kernel_regularizer=None,  \
                              bias_regularizer=None, activity_regularizer=None,   \
                              kernel_constraint=None, bias_constraint=None))

model.add(Dense(units=100))
model.add(Activation('relu'))

model.add(keras.layers.Conv2DTranspose(filters=deconv_f, kernel_size=deconv_k,    \
                                       strides=deconv_s, padding='valid',         \
                                       data_format=None, activation=None,         \
                                       use_bias=True,                             \
                                       kernel_initializer='glorot_uniform',       \
                                       bias_initializer='zeros',                  \
                                       kernel_regularizer=None,                   \
                                       bias_regularizer=None,                     \
                                       activity_regularizer=None,                 \
                                       kernel_constraint=None,                    \
                                       bias_constraint=None))

Using TensorFlow backend.
